In [2]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader.data as pdr
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
import datetime

from prophet import Prophet

C:\Users\0074fasaet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
# download stocks
file = pd.read_csv('M6_Universe.csv')
stocks_list = file['symbol'].values.tolist()

end = datetime.datetime.now()
start = datetime.datetime(end.year - 3, end.month, end.day)

last_month_days = []
for year in range(start.year, end.year+1):
    for month in range(1,13):
        first_day = datetime.datetime(year, month, 1)
        prev_day = first_day - datetime.timedelta(days=1)
        last_month_days += [prev_day.strftime('%Y-%m-%d')]

## Download 100 assets

In [4]:
all_stocks = DataReader(stocks_list, 'yahoo', start, end)['Adj Close']
spy = DataReader(['SPY'], 'yahoo', start, end)['Adj Close']

marketCaps = pdr.get_quote_yahoo(stocks_list)['marketCap']
all_stocks.describe()

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,...,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,104.692022,261.917682,84.402679,139.330084,114.227161,95.988888,211.665907,141.128055,190.961080,159.728946,...,31.066860,114.011332,50.180498,154.110392,87.112114,63.818884,62.628705,65.381243,70.569295,62.647533
std,27.121856,61.422265,8.034546,25.494880,14.609175,35.675868,63.996017,29.701189,34.149013,39.748304,...,6.042559,16.320101,15.943255,28.085965,13.946514,11.530829,6.374604,7.222648,13.308115,45.846006
min,57.469624,138.960983,64.298126,76.798347,76.430473,37.084084,78.741379,83.830498,111.204712,75.713036,...,16.901663,72.100327,21.039660,85.628128,47.169888,39.391159,41.872681,45.845875,36.707245,17.629999
25%,81.788933,205.402378,78.937841,119.826017,100.703779,59.501868,151.562706,115.775625,157.491135,122.691448,...,26.184786,99.255253,36.363009,127.607975,76.402420,54.488731,58.042198,59.202602,57.519330,25.000000
50%,101.565769,260.542984,83.136566,137.304070,112.661373,99.060844,221.095284,154.467499,194.992302,164.568436,...,31.775001,114.425106,48.827709,156.352165,88.326099,61.212975,62.425137,64.761799,74.138512,52.639999
75%,123.194540,312.494545,87.674509,159.097713,127.622448,131.340660,267.922646,164.633247,217.981827,198.866158,...,36.604440,129.681053,55.956556,176.754547,99.975286,75.338514,66.562645,71.388512,82.361933,88.619999
max,171.877823,411.651428,105.180000,191.601440,146.209808,165.868637,324.398621,186.570496,254.599045,223.893570,...,41.039799,142.229568,91.256622,210.293747,106.228661,85.302856,78.120003,79.969498,89.927559,276.000000


In [5]:
# extract monthly returns
month_stocks = all_stocks.copy()
month_stocks['Datestring'] = month_stocks.index
    
month_stocks['year'] = pd.DatetimeIndex(month_stocks.index).year
month_stocks['month'] = pd.DatetimeIndex(month_stocks.index).month

month_on_month = month_stocks.sort_values(by='Date').groupby(['year','month']).last()
month_on_month = month_on_month.drop(month_on_month.tail(1).index).set_index(['Datestring'])

monthly_returns = month_on_month.pct_change().round(3).T
monthly_returns

Datestring,2019-09-30,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,2020-04-30,2020-05-29,2020-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31
Symbols,,,,,,,,,,,,,,,,,,,,,
ABBV,NaN,0.066,0.103,0.009,-0.073,0.058,-0.111,0.095,0.127,0.059,...,0.005,0.175,0.021,0.079,0.097,-0.086,0.003,0.039,-0.054,-0.063
ACN,NaN,-0.032,0.085,0.047,-0.022,-0.120,-0.096,0.139,0.089,0.065,...,-0.004,0.160,-0.145,-0.106,0.067,-0.107,-0.006,-0.070,0.107,-0.058
AEP,NaN,0.007,-0.025,0.035,0.103,-0.138,-0.104,0.039,0.035,-0.066,...,-0.034,0.098,0.016,0.012,0.101,-0.007,0.038,-0.060,0.027,0.025
AIZ,NaN,0.002,0.059,-0.013,-0.004,-0.072,-0.137,0.021,-0.028,0.007,...,-0.053,0.025,-0.021,0.117,0.071,0.000,-0.025,-0.022,0.017,-0.095
ALLE,NaN,0.120,0.034,0.040,0.038,-0.111,-0.197,0.093,-0.008,0.028,...,-0.036,0.074,-0.073,-0.067,-0.038,0.041,-0.023,-0.120,0.080,-0.100
AMAT,NaN,0.087,0.071,0.054,-0.050,0.005,-0.212,0.084,0.135,0.076,...,0.079,0.069,-0.122,-0.027,-0.018,-0.163,0.065,-0.224,0.165,-0.110
AMP,NaN,0.026,0.093,0.017,-0.007,-0.141,-0.275,0.122,0.229,0.071,...,-0.038,0.042,0.009,-0.011,0.002,-0.116,0.045,-0.140,0.136,-0.002
AMZN,NaN,0.023,0.014,0.026,0.087,-0.062,0.035,0.269,-0.013,0.130,...,0.040,-0.049,-0.103,0.027,0.061,-0.238,-0.033,-0.116,0.271,-0.061
AVB,NaN,0.011,-0.015,-0.015,0.033,-0.074,-0.259,0.107,-0.043,0.002,...,0.009,0.064,-0.033,-0.023,0.047,-0.084,-0.086,-0.058,0.101,-0.061


In [13]:
# Generate monthly return forecast table
future_steps = 4
forecast_df_dict = {}
for ci in [0.80, 0.60]:
    print('CI: %f' % ci)
    first_time = True
    for ticker in stocks_list:
        print('Ticker: %s...' % ticker)
        df = monthly_returns.T[ticker].dropna().reset_index().rename(columns={'Datestring':'ds', ticker:'y'})
        model = Prophet(interval_width=ci).fit(df)
        future_dates = model.make_future_dataframe(periods=future_steps, freq='M')
        forecast = model.predict(future_dates)
        if first_time:
            forecast_df = forecast[['ds']].set_index('ds')
            lower_df = forecast_df.copy()
            yhat_df = forecast_df.copy()
            upper_df = forecast_df.copy()
            
            first_time = False
            
        forecast = forecast.set_index('ds')
        
        lower_df[ticker] = forecast['yhat_lower']
        yhat_df[ticker] = forecast['yhat']
        upper_df[ticker] = forecast['yhat_upper']
        forecast_df_dict[ci] = (lower_df, yhat_df, upper_df)
        

09:47:53 - cmdstanpy - INFO - Chain [1] start processing


CI: 0.800000
Ticker: ABBV...


09:47:53 - cmdstanpy - INFO - Chain [1] done processing
09:47:54 - cmdstanpy - INFO - Chain [1] start processing


Ticker: ACN...


09:47:54 - cmdstanpy - INFO - Chain [1] done processing
09:47:54 - cmdstanpy - INFO - Chain [1] start processing


...

09:49:45 - cmdstanpy - INFO - Chain [1] done processing
09:49:45 - cmdstanpy - INFO - Chain [1] start processing


Ticker: XLB...


09:49:45 - cmdstanpy - INFO - Chain [1] done processing
09:49:46 - cmdstanpy - INFO - Chain [1] start processing


Ticker: VXX...


09:49:46 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
forecast_df_dict[0.8][2]

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-10-31,0.081219,0.026158,0.090653,0.055893,0.141176,0.142755,0.108653,0.078623,0.031191,0.163462,...,0.071959,0.047595,0.074980,0.040824,0.058085,0.061892,0.054417,0.014141,0.043558,0.172704
2019-11-29,0.148606,0.113297,0.028525,0.106939,0.089720,0.108422,0.131019,0.068870,0.024013,0.071466,...,0.077266,0.072915,0.064298,0.040640,0.081842,0.054796,0.014197,0.042135,0.060507,0.139135
2019-12-31,0.090569,0.081174,0.086353,0.026025,0.105525,0.071269,0.029073,0.064431,0.017660,0.062641,...,0.037004,0.066184,0.040448,0.024194,0.029944,0.035137,0.066277,0.069059,0.045347,0.269856
2020-01-31,0.039925,0.004002,0.077052,0.047622,0.059306,0.084992,0.102869,0.104361,0.064813,0.046345,...,0.055553,0.006372,0.135048,0.044511,0.045416,0.051171,0.045737,0.023774,0.015051,0.397167
2020-02-28,0.133999,0.006906,-0.022286,0.067932,0.028848,0.163764,0.063251,0.099400,0.038214,-0.003961,...,0.042724,0.014346,0.089750,0.039680,0.043206,0.028503,-0.017658,-0.003486,0.040316,0.405109
2020-03-31,-0.004898,0.048328,0.041081,0.028941,0.024506,0.030714,-0.069017,0.043446,-0.046655,0.052525,...,-0.042008,0.034798,-0.100301,-0.017517,-0.007096,-0.016276,0.029205,0.031873,0.006870,0.696546
2020-04-30,0.148171,0.184825,0.100171,0.117535,0.173409,0.164577,0.224592,0.301042,0.164563,0.201783,...,0.166349,0.139113,0.291100,0.212894,0.142923,0.178521,0.092134,0.084723,0.179820,0.024683
2020-05-29,0.114955,0.097497,0.049919,0.053275,0.106997,0.212534,0.197839,0.075497,0.041132,0.069020,...,0.103662,0.060646,0.157682,0.077010,0.101271,0.100771,0.052593,0.036237,0.117204,0.113358
2020-06-30,0.108374,0.120747,0.004685,0.042536,0.106780,0.164740,0.130862,0.184861,0.071998,0.082395,...,0.047110,0.024782,0.092294,0.099706,0.075349,0.060045,0.012199,0.034988,0.047648,0.174454


In [15]:
forecast_df_dict[0.6][2]

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-10-31,0.061546,0.003963,0.071755,0.037786,0.124104,0.121933,0.085431,0.058895,0.008223,0.145661,...,0.049339,0.031862,0.038237,0.023980,0.035475,0.049865,0.041168,0.000657,0.028457,0.119674
2019-11-29,0.124251,0.095969,0.006755,0.085246,0.064302,0.079396,0.105844,0.045095,0.003362,0.049141,...,0.061031,0.061362,0.019989,0.027730,0.063278,0.041958,0.000563,0.029739,0.046127,0.049228
2019-12-31,0.069914,0.062427,0.070727,0.009848,0.083865,0.048780,0.005175,0.040293,-0.003582,0.039247,...,0.016865,0.054089,0.008921,0.009119,0.016374,0.025239,0.054242,0.058350,0.033057,0.169863
2020-01-31,0.021911,-0.015760,0.064751,0.025871,0.037526,0.049703,0.080911,0.087949,0.043314,0.027674,...,0.034503,-0.003535,0.102854,0.028613,0.025984,0.038431,0.034883,0.010849,-0.003765,0.309168
2020-02-28,0.111669,-0.014749,-0.038826,0.044213,-0.001383,0.135534,0.043649,0.082796,0.017696,-0.025965,...,0.021448,0.002384,0.063057,0.024534,0.028655,0.017941,-0.033378,-0.013794,0.022273,0.317323
2020-03-31,-0.027359,0.034995,0.019235,0.006489,0.001028,0.001476,-0.092075,0.026250,-0.069148,0.029605,...,-0.063840,0.026415,-0.144112,-0.031311,-0.022306,-0.027126,0.014148,0.020788,-0.006938,0.595860
2020-04-30,0.130503,0.167651,0.080445,0.103287,0.151240,0.130439,0.197235,0.281086,0.144081,0.179687,...,0.143996,0.126239,0.256348,0.196771,0.125446,0.164389,0.076985,0.072769,0.161482,-0.068550
2020-05-29,0.093975,0.081519,0.033893,0.035672,0.078061,0.188707,0.171819,0.055702,0.022322,0.050054,...,0.080821,0.048701,0.118591,0.056437,0.083216,0.086379,0.040460,0.024072,0.096681,0.030402
2020-06-30,0.087788,0.102629,-0.012543,0.021139,0.084749,0.142253,0.104532,0.161328,0.056660,0.064455,...,0.024847,0.010055,0.052573,0.083249,0.056118,0.046995,-0.006648,0.022003,0.035513,0.095692


In [16]:
# Quantile table generation
timestamp = '2022-09-30'
quantile_table = pd.DataFrame([],columns=stocks_list).T
quantile_table['20'] = np.nan
quantile_table['40'] = np.nan
quantile_table['y'] = np.nan
quantile_table['60'] = np.nan
quantile_table['80'] = np.nan

quantile_table = quantile_table.T

for ticker in stocks_list:
    q_20 = forecast_df_dict[0.8][0][ticker][timestamp]
    q_40 = forecast_df_dict[0.6][0][ticker][timestamp]
    q_y = forecast_df_dict[0.8][1][ticker][timestamp]
    q_60 = forecast_df_dict[0.6][2][ticker][timestamp]
    q_80 = forecast_df_dict[0.8][2][ticker][timestamp]

    quantile_table[ticker] = [q_20, q_40, q_y, q_60, q_80]
    
    
quantile_table = quantile_table.T
quantile_table

,20,40,y,60,80
ABBV,-0.098724,-0.081666,-0.043585,-0.005385,0.017351
ACN,-0.113884,-0.100094,-0.064416,-0.029041,-0.011605
AEP,-0.108422,-0.095895,-0.060318,-0.029284,-0.010562
AIZ,0.020066,0.033555,0.070806,0.106557,0.125336
ALLE,-0.066858,-0.041300,0.001470,0.047810,0.074446
AMAT,-0.130245,-0.106187,-0.049973,0.000940,0.024018
AMP,-0.101306,-0.080149,-0.036376,0.013478,0.033502
AMZN,-0.184360,-0.163736,-0.125721,-0.086970,-0.071152
AVB,-0.024255,-0.002317,0.040703,0.082202,0.104666
AVY,-0.064452,-0.044207,-0.004488,0.033121,0.051171


In [17]:
# predicted return ranking algorithm
rank1_indices = quantile_table['20'].sort_values(ascending=True).index[:20]
rank2_indices = quantile_table['40'].sort_values(ascending=True).index[20:40]
rank3_indices = quantile_table['y'].sort_values(ascending=True).index[40:60]
rank4_indices = quantile_table['60'].sort_values(ascending=True).index[60:80]
rank5_indices = quantile_table['80'].sort_values(ascending=True).index[80:]

rank_df = quantile_table.copy()
rank_df[:] = 0

rank_df.loc[rank1_indices,'20'] = 1
rank_df.loc[rank2_indices,'40'] = 1
rank_df.loc[rank3_indices,'y'] = 1
rank_df.loc[rank4_indices,'60'] = 1
rank_df.loc[rank5_indices,'80'] = 1

# Set empty rows to carry rank 3
empty_rows = (rank_df.sum(axis=1) == 0)
rank_df.loc[empty_rows, :] = 0.2

rank_df = rank_df.T

# normalize rows
rank_df = rank_df / rank_df.sum()
rank_df = rank_df.T.round(4).rename(columns={'20':'Rank 1','40':'Rank 2','y':'Rank 3','60':'Rank 4','80':'Rank 5'})
rank_df

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5
ABBV,0.0000,1.0000,0.0000,0.0000,0.0000
ACN,1.0000,0.0000,0.0000,0.0000,0.0000
AEP,0.2000,0.2000,0.2000,0.2000,0.2000
AIZ,0.0000,0.0000,0.0000,0.0000,1.0000
ALLE,0.0000,0.0000,0.0000,1.0000,0.0000
AMAT,1.0000,0.0000,0.0000,0.0000,0.0000
AMP,0.0000,1.0000,0.0000,0.0000,0.0000
AMZN,1.0000,0.0000,0.0000,0.0000,0.0000
AVB,0.0000,0.0000,0.0000,0.5000,0.5000
AVY,0.0000,0.0000,1.0000,0.0000,0.0000


In [19]:
#portfolio strategy
strategy = rank_df['Rank 5'] / rank_df['Rank 5'].sum()
strategy = strategy.round(5)
strategy

ABBV      0.00000
ACN       0.00000
AEP       0.01007
AIZ       0.05034
ALLE      0.00000
AMAT      0.00000
AMP       0.00000
AMZN      0.00000
AVB       0.02517
AVY       0.00000
AXP       0.05034
BDX       0.00000
BF-B      0.00000
BMY       0.00000
BR        0.00000
CARR      0.00000
CDW       0.00000
CE        0.05034
CHTR      0.00000
CNC       0.00000
CNP       0.05034
COP       0.05034
CTAS      0.00000
CZR       0.05034
DG        0.00000
DPZ       0.00000
DRE       0.00000
DXC       0.05034
META      0.00000
FTV       0.00000
GOOG      0.00000
GPC       0.02517
HIG       0.05034
HST       0.05034
JPM       0.01007
KR        0.00000
OGN       0.01678
PG        0.00000
PPL       0.00000
PRU       0.05034
PYPL      0.00000
RE        0.00000
ROL       0.00000
ROST      0.00000
UNH       0.00000
URI       0.05034
V         0.01007
VRSK      0.00000
WRK       0.05034
XOM       0.05034
IVV       0.01007
IWM       0.00000
EWU       0.00000
EWG       0.00000
EWL       0.00000
EWQ       

In [20]:
# Save to file
submission = rank_df.copy()

diff = 1 - submission[['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']].sum(1)
submission['Rank 5'] = submission['Rank 5'] + diff

diff = 1 - strategy.sum()
strategy[-1] = strategy[-1] + diff

submission['Decision'] = strategy
submission = submission.round(5)
submission.to_csv('submission.csv')
submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.0000,1.0000,0.0000,0.0000,0.0000,0.00000
ACN,1.0000,0.0000,0.0000,0.0000,0.0000,0.00000
AEP,0.2000,0.2000,0.2000,0.2000,0.2000,0.01007
AIZ,0.0000,0.0000,0.0000,0.0000,1.0000,0.05034
ALLE,0.0000,0.0000,0.0000,1.0000,0.0000,0.00000
AMAT,1.0000,0.0000,0.0000,0.0000,0.0000,0.00000
AMP,0.0000,1.0000,0.0000,0.0000,0.0000,0.00000
AMZN,1.0000,0.0000,0.0000,0.0000,0.0000,0.00000
AVB,0.0000,0.0000,0.0000,0.5000,0.5000,0.02517
AVY,0.0000,0.0000,1.0000,0.0000,0.0000,0.00000
